In [ ]:
import requests 
import pandas as pd
import os
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import json
from lxml import etree
from urllib.parse import urljoin

In [ ]:
class Drugs_links_collecter():
    def __init__(self, start_char = 'a', end_char = 'z', drugs_char_childs_urls = {}, durgs_urls = {}):
        self.base_url = 'https://www.drugs.com/'
        self.start_char = start_char
        self.end_char = end_char
        
        self.drugs_char_childs_urls = drugs_char_childs_urls
        self.durgs_urls = durgs_urls
        
        
    def reset_maxout(self):
        self.request_maxout = 5
        
        
    def get_dom(self, url):
        self.reset_maxout()
        while self.request_maxout != 0:
            res = requests.get(url)
            if res.status_code == 200: 
                html = BeautifulSoup(res.text, 'html.parser') 
                dom = etree.HTML(str(html))
                return dom
            else:
                print(res.status_code)
                self.request_maxout -= 1
        raise Exception("couldn't request the needed page")
        
    def get_char_childs(self):
        for i in range(ord('a'), ord('z')+1):
            char = chr(i)
            self.drugs_char_childs_urls[char] = {}
            print('getting childs of drugs starting with ',char)
            dom = self.get_dom(self.base_url  + 'alpha/'+ char+'.html')
            time.sleep(3)
            p = '//*[@id="content"]/div[2]/nav[1]/ul/li'
            childs = dom.xpath(p)
            childs_xpath = p + '[?]'
    
            for j in range(len(childs_xpath)):
                xpath = childs_xpath.replace("?",str(j))+"/a"
                a_tags = dom.xpath(xpath)
                if len(a_tags) > 0:
                    a_tag = a_tags[0]
                    name = a_tag.text
                    url = a_tag.get("href")
                    self.drugs_char_childs_urls[char][name] =  url
    
    def get_char_urls(self, char):
        for char_child in self.drugs_char_childs_urls[char].keys():
            url = urljoin(self.base_url, self.drugs_char_childs_urls[char][char_child])
            time.sleep(3)
            dom = self.get_dom(url)
            drugs_list_xpath = '//*[@id="content"]/div[2]/ul/li'
            drugs_list = dom.xpath(drugs_list_xpath)
            drug_xpath = drugs_list_xpath + '[?]'
            for i in range(len(drugs_list)):
                xpath = drug_xpath.replace("?",str(i))+"/a"
                a_tags = dom.xpath(xpath)
                if len(a_tags) > 0:
                    a_tag = a_tags[0]
                    name = a_tag.text
                    url = a_tag.get("href")
                    self.durgs_urls[name] = urljoin(self.base_url, url)
                    
    def get_all_drugs(self):
        for i in range(ord(self.start_char), ord(self.end_char)+1):
            print('getting all drugs starting with ',chr(i))
            self.get_char_urls(chr(i))
    
    def save_json(self, path = None):
        for i, d in zip(['drugs_pages_urls', 'drugs_urls'], [self.drugs_char_childs_urls, self.durgs_urls]):
            if path is not None:
                full_path = os.path.join(path, + i +".json")
            else:
                full_path = i + ".json"

            jsonString = json.dumps(d)
            jsonFile = open(full_path, "w")
            jsonFile.write(jsonString)
            jsonFile.close()

In [ ]:
dlc = Drugs_links_collecter()

In [ ]:
dlc.get_char_childs()

In [ ]:
dlc.drugs_char_childs_urls

In [ ]:
dlc.get_all_drugs()

In [ ]:
len(dlc.durgs_urls.keys())

In [ ]:
d = dlc.durgs_urls
jsonString = json.dumps(d)
jsonFile = open('./drugs/drugs_urls.json', "w")
jsonFile.write(jsonString)
jsonFile.close()

In [ ]:
with open('./drugs/drugs_urls.json') as handle:
    d = json.loads(handle.read())


In [ ]:
len(d.keys())

In [ ]:
d = dlc.drugs_char_childs_urls
jsonString = json.dumps(d)
jsonFile = open('./drugs/drugs_pages_urls.json', "w")
jsonFile.write(jsonString)
jsonFile.close()